In [1]:
import configparser, time, random, string

from azure.servicebus import ServiceBusService, Message, Queue, Topic, Rule, DEFAULT_RULE_NAME


In [2]:
uk_area_codes=['AA', 'AB', 'AC', 'AD', 'AE', 'AF', 'AG', 'AH', 'AJ', 'AK', 'AL', 'AM', 'AN', 'AO', 'AP', 'AR', 'AS', 'AT', 'AU', 'AV', 'AW', 'AX', 'AY', 'BA', 'BB', 'BC', 'BD', 'BE', 'BF', 'BG', 'BH', 'BJ', 'BK', 'BL', 'BM', 'BN', 'BO', 'BP', 'BR', 'BS', 'BT', 'BU', 'BV', 'BW', 'BX', 'BY', 'CA', 'CB', 'CC', 'CD', 'CE', 'CF', 'CG', 'CH', 'CJ', 'CK', 'CL', 'CM', 'CN', 'CO', 'CP', 'CR', 'CS', 'CT', 'CU', 'CV', 'CW', 'CX', 'CY', 'DA', 'DB', 'DC', 'DD', 'DE', 'DF', 'DG', 'DH', 'DJ', 'DK', 'DL', 'DM', 'DN', 'DO', 'DP', 'DR', 'DS', 'DT', 'DU', 'DV', 'DW', 'DX', 'DY', 'EA', 'EB', 'EC', 'ED', 'EE', 'EF', 'EG', 'EH', 'EJ', 'EK', 'EL', 'EM', 'EN', 'EO', 'EP', 'ER', 'ES', 'ET', 'EU', 'EV', 'EW', 'EX', 'EY', 'FA', 'FB', 'FC', 'FD', 'FE', 'FF', 'FG', 'FH', 'FJ', 'FK', 'FL', 'FM', 'FN', 'FP', 'FR', 'FS', 'FT', 'FV', 'FW', 'FX', 'FY', 'GA', 'GB', 'GC', 'GD', 'GE', 'GF', 'GG', 'GH', 'GJ', 'GK', 'GL', 'GM', 'GN', 'GO', 'GP', 'GR', 'GS', 'GT', 'GU', 'GV', 'GX', 'GY', 'HA', 'HB', 'HC', 'HD', 'HE', 'HF', 'HG', 'HH', 'HJ', 'HK', 'HL', 'HM', 'HN', 'HO', 'HP', 'HR', 'HS', 'HT', 'HU', 'HV', 'HW', 'HX', 'HY', 'KA', 'KB', 'KC', 'KD', 'KE', 'KF', 'KG', 'KH', 'KJ', 'KK', 'KL', 'KM', 'KN', 'KO', 'KP', 'KR', 'KS', 'KT', 'KU', 'KV', 'KW', 'KX', 'KY', 'LA', 'LB', 'LC', 'LD', 'LE', 'LF', 'LG', 'LH', 'LJ', 'LK', 'LL', 'LM', 'LN', 'LO', 'LP', 'LR', 'LS', 'LT', 'LU', 'LV', 'LW', 'LX', 'LY', 'MA', 'MB', 'MC', 'MD', 'ME', 'MF', 'MG', 'MH', 'MJ', 'MK', 'ML', 'MM', 'MN', 'MO', 'MP', 'MR', 'MS', 'MT', 'MU', 'MV', 'MW', 'MX', 'MY', 'NA', 'NB', 'NC', 'ND', 'NE', 'NF', 'NG', 'NH', 'NJ', 'NK', 'NL', 'NM', 'NN', 'NO', 'NP', 'NR', 'NS', 'NT', 'NU', 'NV', 'NW', 'NX', 'NY', 'OA', 'OB', 'OC', 'OD', 'OE', 'OF', 'OG', 'OH', 'OJ', 'OK', 'OL', 'OM', 'ON', 'OO', 'OP', 'OR', 'OS', 'OT', 'OU', 'OV', 'OW', 'OX', 'OY', 'PA', 'PB', 'PC', 'PD', 'PE', 'PF', 'PG', 'PH', 'PJ', 'PK', 'PL', 'PM', 'PN', 'PO', 'PP', 'PR', 'PS', 'PT', 'PU', 'PV', 'PW', 'PX', 'PY', 'RA', 'RB', 'RC', 'RD', 'RE', 'RF', 'RG', 'RH', 'RJ', 'RK', 'RL', 'RM', 'RN', 'RO', 'RP', 'RR', 'RS', 'RT', 'RU', 'RV', 'RW', 'RX', 'RY', 'SA', 'SB', 'SC', 'SD', 'SE', 'SF', 'SG', 'SH', 'SJ', 'SK', 'SL', 'SM', 'SN', 'SO', 'SP', 'SR', 'SS', 'ST', 'SU', 'SV', 'SW', 'SX', 'SY', 'VA', 'VB', 'VC', 'VD', 'VE', 'VF', 'VG', 'VH', 'VJ', 'VK', 'VL', 'VM', 'VN', 'VO', 'VP', 'VR', 'VS', 'VT', 'VU', 'VV', 'VW', 'VX', 'VY', 'WA', 'WB', 'WC', 'WD', 'WE', 'WF', 'WG', 'WH', 'WJ', 'WK', 'WL', 'WM', 'WN', 'WO', 'WP', 'WR', 'WS', 'WT', 'WU', 'WV', 'WW', 'WX', 'WY', 'YA', 'YB', 'YC', 'YD', 'YE', 'YF', 'YG', 'YH', 'YJ', 'YK', 'YL', 'YM', 'YN', 'YO', 'YP', 'YR', 'YS', 'YT', 'YU', 'YV', 'YW', 'YX', 'YY']

In [3]:
# Linking to the Service Bus

bus_service=ServiceBusService(
    service_namespace='Dream22',
    shared_access_key_name='RootManageSharedAccessKey',
    shared_access_key_value='eykA9ft6Ghg621smEDZK8uc0Y38EtgWBNZ6pCMzebvA=')

In [4]:
# Reading in the config file

Config=configparser.ConfigParser()
Config
Config.read("config.ini")
Config.sections()

['SectionOne', 'SectionTwo']

In [5]:
# Setting up the function to extract variables from the config file

def ConfigSectionMap(section):
    dict1={}
    options=Config.options(section)
    for option in options:
        try:
            dict1[option] = Config.get(section,option)
            if dict1[option]==-1:
                DebugPrint("skip: %s" % option)

        except:
            print("exception on %s!" % option)
            dict1[option]=None
    return dict1

In [6]:
# Reading from config file

camera = ConfigSectionMap("SectionOne")['id']
street = ConfigSectionMap("SectionOne")['street name']
town = ConfigSectionMap("SectionOne")['town/city']
speedlimit = int(ConfigSectionMap("SectionOne")['max speed'])
rate = ConfigSectionMap("SectionTwo")['rate']


cars=[]

In [7]:
# Create topics for cameras and cars
#bus_service.delete_topic('camera_switchons')
#bus_service.create_topic('camera_switchons')
#bus_service.create_topic('car_sightings')

# Create subscriptions
#bus_service.delete_subscription('camera_switchons', 'CameraSwitchons')
#bus_service.create_subscription('camera_switchons', 'CameraSwitchons')
#bus_service.delete_subscription('car_sightings', 'CarSightings')
#bus_service.create_subscription('car_sightings', 'CarSightings')

In [11]:
# Sending camera switchon message to topic

switchon = Message(bytes({0}),custom_properties={"id":camera,"street":street,"town":town,"speedlimit":speedlimit,"time":time.strftime("%H.%M.%S-%d.%m.%Y")})

bus_service.send_topic_message('camera_switchons',switchon)

In [12]:
# Launching simulation of cars
 
period=60/int(rate)

localstorage=[]

while True:

    registration=random.choice(uk_area_codes)+str(random.randint(0,9))+str(random.randint(0,9))+' '+random.choice(string.ascii_uppercase)+random.choice(string.ascii_uppercase)+random.choice(string.ascii_uppercase)
    vtype=random.choice(['Car','Truck','Motorcycle'])
    speed=random.randint(speedlimit-20,speedlimit+20)
    msg = Message(bytes({0}),custom_properties={"cameraid":camera,"street":street,"town":town,"speedlimit":speedlimit,"registration":registration,"type":vtype,"speed":speed})
    try: 
        bus_service.send_topic_message(('car_sightings'), msg)
        print('Sent on generation')
        print(msg.custom_properties['registration'])
        print(msg.custom_properties['cameraid'])
        print(msg.custom_properties['speed'])
        print('----')
        print()
        
        if localstorage!=[]:
            for message in localstorage:
                
                try:
                    bus_service.send_topic_message(('car_sightings'), message)
                    
                    localstorage.remove(message)
                    
                    print('Sent from local storage')
                    print(message.custom_properties['registration'])
                    print(message.custom_properties['cameraid'])
                    print(message.custom_properties['speed'])
                    print('----')
                    print()
                    
                except:
                    break
                
                
    except:
        localstorage.append(msg)
        print('Saved in local storage')
        print(msg.custom_properties['registration'])
        print(msg.custom_properties['cameraid'])
        print(msg.custom_properties['speed'])
        print('----')
        print()
        
    
    time.sleep(period)

    

Sent on generation
HS08 UJU
46638
34
----

Sent on generation
SE33 OHW
46638
36
----

Sent on generation
KB53 BES
46638
26
----

Sent on generation
ER37 EZJ
46638
56
----

Sent on generation
LA84 IMS
46638
33
----

Sent on generation
KS86 PNI
46638
30
----

Sent on generation
MB53 OLO
46638
25
----

Sent on generation
WA78 FHT
46638
56
----

Sent on generation
CN60 VTT
46638
57
----

Sent on generation
FX21 FAX
46638
35
----

Sent on generation
YH16 PLU
46638
58
----

Sent on generation
GV22 ZII
46638
55
----

Sent on generation
FM25 DHK
46638
55
----

Sent on generation
SG80 ZGY
46638
29
----

Sent on generation
YP47 IFT
46638
57
----



KeyboardInterrupt: 